In [20]:
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import os


In [21]:
path = os.getcwd()

In [22]:
# Load datasets
train_df = pd.read_csv(path+'/data/train.csv')

# Preprocessing
X = train_df['TEXT'].fillna("blank").astype(str)
y = train_df['LABEL']

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [23]:
# Initialize vectorizers
tfidf_vect = TfidfVectorizer(ngram_range= (1,3), lowercase = True)

# Fit and transform the train data
X_train_tfidf = tfidf_vect.fit_transform(X_train)

# Transform the test data
X_test_tfidf = tfidf_vect.transform(X_val)


# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder on the labels
label_encoder.fit(y_train)

# Transform labels to numeric values
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_val)


In [24]:
from sklearn.naive_bayes import MultinomialNB

class Classifiers:
    def __init__(self):
        self.logistic = LogisticRegression(max_iter = 1000, C = 3)
        self.bayes = MultinomialNB()
        self.svm = LinearSVC(max_iter = 1000, C = 3)

    def fit(self, features, labels):
        self.logistic.fit(features,labels)
        self.bayes.fit(features,labels)
        self.svm.fit(features,labels)


    def predict(self, features):
        return self.logistic.predict(features), self.bayes.predict(features), self.svm.predict(features)

In [25]:
clf = Classifiers()

clf.fit(X_train_tfidf, y_train_encoded)
logistics, bayes,svm = clf.predict(X_test_tfidf)

In [29]:
print("Logistic Regressions:", '=' * 50)
print(classification_report(y_val, logistics, target_names = ['Not a movie', "Positive review", "Negative review"]))

print("Naive Bayes Model:", '=' * 50)
print(classification_report(y_val, bayes, target_names = ['Not a movie', "Positive review", "Negative review"]))

print("Linear SVC:", '=' * 50)
print(classification_report(y_val, svm, target_names = ['Not a movie', "Positive review", "Negative review"]))

Logistic Regressions: ==================================================
                 precision    recall  f1-score   support

    Not a movie       0.97      0.97      0.97      6454
Positive review       0.87      0.90      0.88      3856
Negative review       0.91      0.88      0.89      3754

       accuracy                           0.93     14064
      macro avg       0.92      0.91      0.91     14064
   weighted avg       0.93      0.93      0.93     14064

Naive Bayes Model: ==================================================
                 precision    recall  f1-score   support

    Not a movie       0.97      0.94      0.96      6454
Positive review       0.86      0.85      0.85      3856
Negative review       0.85      0.91      0.88      3754

       accuracy                           0.91     14064
      macro avg       0.89      0.90      0.90     14064
   weighted avg       0.91      0.91      0.91     14064

Linear SVC: =========================================

In [30]:
test_df = pd.read_csv(path + '/data/test.csv')
test_df = test_df.fillna('blank')
test_tfidf = tfidf_vect.transform(test_df['TEXT'])

_,_,predictions = clf.predict(test_tfidf)

In [31]:
submission = pd.DataFrame(columns=['ID','LABEL'])
submission['ID'] = test_df['ID']
submission['LABEL'] = predictions

submission

,ID,LABEL
0,4728459160322025755,1
1,1840432070229003467,1
2,12623336783082722606,2
3,7446733850828603409,0
4,16180660281866613068,2
...,...,...
17575,15460118162570972562,2
17576,2679547768967862209,0
17577,2966026531113989116,0
17578,10698695044532313190,0


In [32]:
submission.to_csv(path + '/submission_v6.csv', index = False)